In [ ]:
!pip install transformers

In [ ]:
import transformers
import torch
import torch.nn.functional as F
import numpy as np
import random
device = "cpu"
model_name ="TrustSafeAI/RADAR-Vicuna-7B"
detector = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
detector.eval()
detector.to(device)

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [ ]:
def get_ans(Text_input):
  with torch.no_grad():
    inputs = tokenizer(Text_input, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    output_probs = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()

  label = ''
  label_confidence = 0

  if output_probs[0] > 0.5:
    label = 'AI-generated'
    label_confidence = output_probs[0]
  else:
    label = 'Human-written'
    label_confidence = 1 - output_probs[0]

  return label, label_confidence

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
# Read dataset file
df = pd.read_csv('')
df.head(5)

In [ ]:
start_index =    # Set your desired start index
end_index =   # Set your desired end index
total_to_process = end_index - start_index

for index, row in tqdm(enumerate(df.iterrows(), start=start_index), total=total_to_process, desc="Processing rows"):
    if start_index <= index < end_index:
        article = row[1]['GPT4_responses']
        text_input =[article]
        label, confidence = get_ans(text_input)

        df.loc[index, 'label'] = label
        df.loc[index, 'confidence'] = confidence

In [ ]:
# Save file
df.to_csv('', index=False)